# Automated ML

Import Dependencies. 

In [1]:
from azureml.core import Workspace, Experiment, Dataset
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails
from azureml.core.run import Run
import pandas as pandas
from azureml.data.datapath import DataPath
from azureml.train.automl import AutoMLConfig
import joblib
import os
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException


## Dataset

### Overview
This experiment with AutoML uses a dataset from Kaggle regarding heart failures with 12 features to predict whether an event of death occurs or not. Therefore, this is a classification prediction task.

In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'automl'

experiment=Experiment(ws, experiment_name)

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code FETQUPAKW to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.


In [3]:
ds = Dataset.get_by_name(ws, 'dataset')

In [4]:
df = ds.to_pandas_dataframe()
df.head()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1


In [5]:
train_data, test_data = ds.random_split(0.7)

## AutoML Configuration

In the AutoML configuration the following parameters are applied:
- experiment_timeout_minutes:30, The number of minutes the experiment should stay running.
- primary_metric:'accuracy', The primary metric parameter for optimization.
- n_cross_validations:5, As the dataset is relatively small, a low number of number of cross validations is applied.
- max_concurrent_iterations: 4, To get a dedicated cluster per experiment, the max_concurrent_iterations is set at four, to manage the child runs and when they can be performed. Four max_concurrent_iterations is chosen, as the number of nodes in the cluster is also four.
- training_data=train_data, The train_data is part of the dataset uploaded for this experiment.
- label_column_name='DEATH_EVENT', The target column is death_event, as we want to predict whether a patient passes away.
- featurization='auto', Automatic guardrails and featurization steps part of preprocessing.
- compute_target=cpu_cluster, The ML model is trained on a cluster of Azure virtual machines from Azure Machine Learning Managed Compute. 'STANDARD_D2_V2' with a maximum of 4 nodes in a cpu_cluster is used as compute_target.
- task = 'classification', As the goal is to predict whether someone dies or not, this experiment becomes a classification task type.

In [6]:
cpu_cluster_name = "computecluster"

# Verify if the clusters already exists
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing target, use it')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size = 'STANDARD_D2_V2', max_nodes = 4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output = True)

Found existing target, use it
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [7]:
#Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes":30,
    "primary_metric":'accuracy',
    "n_cross_validations":5,
    "max_concurrent_iterations": 4}

# Put your automl config here
automl_config = AutoMLConfig(training_data=train_data,
label_column_name='DEATH_EVENT',
featurization='auto',
    compute_target=cpu_cluster,
    task = 'classification',
    **automl_settings)

In [8]:
# Submit your experiment
remote_run = experiment.submit(automl_config, show_output = True)

Running on remote.
No run_configuration provided, running on computecluster with default configuration
Running on remote compute: computecluster
Parent Run ID: AutoML_013c9a3c-3c89-4a0c-aa19-3106943a7d4e

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputa

## Run Details

`RunDetails` widget to show the different experiments.

In [ ]:
RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=True)

## Best Model

The best model from the automl experiments and display all the properties of the model.



In [ ]:
# Retrieve and save your best automl model.
best_run, best_model = remote_run.get_output()
print(best_run)

In [ ]:
best_run

In [ ]:
print(best_model)


In [ ]:
#TODO: Save the best model
automl_model = best_run.register_model(model_name = 'best_automl_model.pkl', model_path = './outputs/')

In [ ]:
cpu_cluster.delete()

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service